In [702]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import torch
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input, Embedding, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.neighbors import KNeighborsClassifier




**Computational Tool Final Project**


In [79]:
books = pd.read_csv('file.csv')
#clean_books1 = books[["Title","description","authors","publishedDate","categories","averageRating"]]
clean_books1 = books.dropna()
clean_books1




,Unnamed: 0.1,Unnamed: 0,Title,publishedDate,averageRating,clean_summary,clean_authors,clean_categories
0,0,0,The Church of Christ: A Biblical Ecclesiology ...,1996,4.50,"['church', 'christ', 'biblical', 'ecclesiology...",everett ferguson,religion
1,1,1,Voices from the Farm: Adventures in Community ...,2012-08-21,4.82,"['twenty', 'five', 'years', 'ago', 'height', '...",rupert fike,biography autobiography
2,2,2,The Battleship Bismarck,2018-09-20,3.50,"['bismarck', 'perhaps', 'famous', 'notorious',...",stefan draminski,history
3,3,3,Tess and the Highlander,2002-11,4.24,"['windswept', 'isle', 'scotland', 'seventeen',...",may mcgoldrick,fiction
4,4,4,Beginner's Yoruba (Hippocrene Beginner's Series),2005,3.80,"['beginners', 'yoruba', 'available', 'two', 'a...",kayode j fakinlede,education
...,...,...,...,...,...,...,...,...
41401,41401,41830,Thin Within,2005-04-17,4.93,"['want', 'lose', 'weight', 'dieting', 'doesnt'...",judy halliday arthur halliday,health fitness
41402,41402,41831,The Awakening and Selected Stories (Modern Lib...,2003,3.00,"['first', 'appeared', 'awakening', 'greeted', ...",kate chopin,love stories
41403,41403,41832,Final things,2015-03-17,4.00,"['graces', 'father', 'believes', 'science', 'b...",jenny offill,fiction
41404,41404,41833,The Orphan Of Ellis Island (Time Travel Advent...,2000-06-01,4.68,"['school', 'trip', 'ellis', 'island', 'dominic...",elvira woodruff,fiction


In this project we are going to use a cleaned up version of this data set from Kaggle (https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews?select=books_data.csv&fbclid=IwAR3H0BlyYyiNodMoF0GbzknzNMyriDa_9eL00g1if_Y29QBPcobfeHQpZCE).

This dataset contains two csv files, one containg the ratings and another containg the books. Post cleanup there are 44285 books with  columns 
,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
,Title,description,authors,publishedDate,categories,ratingsCount
* id 
* book name 
* genre 
* summary

The second csv file contains the reviews for the books and has 1502433 reviews total after cleanup (removing the ratings for the books we removed in the cleanup of the first csv file). The columns it has is:
,Id,Title,Price,review/helpfulness,review/score,review/time,review/summary,review/text
,Id,Title,Price,User_id,review/score

Our main goal is to classify the books with text simmilarity ( using summaries) and predict their genre correctly. Moreover, we want to make a simple recomendation UI using github pages which the user can choose between time, rating , genre and get some recomendation based on our classification model.


In [80]:
from functions import * 

clean_books1['clean_summary'] = clean_books1['clean_summary'].apply(lambda x: clean_books(x))
clean_books1['clean_authors'] = clean_books1['clean_authors'].apply(lambda x: clean_books(x))
clean_books1['clean_categories'] = clean_books1['clean_categories'].apply(lambda x: clean_books(x))





/var/folders/wt/pl8gm0gs50d6vpk0nxfh07_h0000gn/T/ipykernel_11813/1834502274.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_books1['clean_summary'] = clean_books1['clean_summary'].apply(lambda x: clean_books(x))
/var/folders/wt/pl8gm0gs50d6vpk0nxfh07_h0000gn/T/ipykernel_11813/1834502274.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_books1['clean_authors'] = clean_books1['clean_authors'].apply(lambda x: clean_books(x))
/var/folders/wt/pl8gm0gs50d6vpk0nxfh07_h0000gn/T/ipykernel_11813/183

In [81]:

clean_books1


,Unnamed: 0.1,Unnamed: 0,Title,publishedDate,averageRating,clean_summary,clean_authors,clean_categories
0,0,0,The Church of Christ: A Biblical Ecclesiology ...,1996,4.50,church christ biblical ecclesiology today res...,everett ferguson,religion
1,1,1,Voices from the Farm: Adventures in Community ...,2012-08-21,4.82,twenty five years ago height counter culture ...,rupert fike,biography autobiography
2,2,2,The Battleship Bismarck,2018-09-20,3.50,bismarck perhaps famous notorious warship eve...,stefan draminski,history
3,3,3,Tess and the Highlander,2002-11,4.24,windswept isle scotland seventeen year old te...,may mcgoldrick,fiction
4,4,4,Beginner's Yoruba (Hippocrene Beginner's Series),2005,3.80,beginners yoruba available two accompanying a...,kayode j fakinlede,education
...,...,...,...,...,...,...,...,...
41401,41401,41830,Thin Within,2005-04-17,4.93,want lose weight dieting doesnt seem work ive...,judy halliday arthur halliday,health fitness
41402,41402,41831,The Awakening and Selected Stories (Modern Lib...,2003,3.00,first appeared awakening greeted cries outrag...,kate chopin,love stories
41403,41403,41832,Final things,2015-03-17,4.00,graces father believes science builds daughte...,jenny offill,fiction
41404,41404,41833,The Orphan Of Ellis Island (Time Travel Advent...,2000-06-01,4.68,school trip ellis island dominick avaro ten y...,elvira woodruff,fiction


In [ ]:
clean_books1['clean_categories'] = clean_books1['clean_categories'].str.split(' ').str[1]

clean_books1

In [26]:

import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 
file1 = pd.read_csv("file.csv") 
for index, row in file1["clean_summary"].iteritems():
    if  isinstance(row, str):
        cat_arr = row.split(" ")
        for word in cat_arr:
            if word in stop_words:
                cat_arr.remove()
        file1['clean_summary'].loc[index] = cat_arr
    
        
        


    
    






/var/folders/wt/pl8gm0gs50d6vpk0nxfh07_h0000gn/T/ipykernel_11813/2253585320.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file1['clean_summary'].loc[index] = cat_arr


In [88]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(clean_books1["clean_summary"])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

<IPython.core.display.Javascript object>

/Users/stef_cm/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


: 

: 

In [82]:
file1.dtypes

Unnamed: 0.1          int64
Unnamed: 0            int64
Title                object
publishedDate        object
averageRating       float64
clean_summary        object
clean_authors        object
clean_categories     object
dtype: object

In [76]:
file1 = pd.read_csv('file.csv')
for index, row in file1["clean_summary"].iteritems():
    if isinstance(row, str):
        row =  clean_books(row)

In [70]:
def listToString(s):
 
    # initialize an empty string
    str1 = ""
 
    # traverse in the string
    for ele in s:
        str1 += ele
 
    # return string
    return str1
       
       
# Driver code   
for index, row in file1["clean_summary"].iteritems():
    if  isinstance(row, str ):
        cat_arr = listToString(row)
        file1['clean_summary'].loc[index] = cat_arr

/var/folders/wt/pl8gm0gs50d6vpk0nxfh07_h0000gn/T/ipykernel_11813/548054310.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file1['clean_summary'].loc[index] = cat_arr


str

In [87]:
clean_books1['clean_summary'].isna().sum()

0

In [1]:
clean_books1.to_csv("file.csv")

NameError: name 'clean_books1' is not defined